# Group 6
- GIL GUZMAN, ALEXSANDER GERARDO
- SALMÓN SALAZAR, GISELLA VELDA
- CIRIACO RUIZ, MAYTE JULIA
- ARAUCO ALIAGA, ROXANA PATRICIA
- VILLACORTA BARRERA, ANDRES ALEXANDER

In [2]:
# We modify the style of the Jupyter Notebook environment
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [ ]:
# We install the different Python packages that we will be using
!pip install selenium
!pip install webdriver-manager
!pip install pandas

In [ ]:
# We use this library to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
import random
from selenium.webdriver.common.by import By
import pandas as pd # for table import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service